In [1]:
import pandas as pd
import os
from datetime import datetime
from item_parser import parse
import sql_con

In [2]:
plytix_prods = pd.read_sql_query("""
    SELECT
        [Артикул],
        [Наименование], 
        [Brand]
    FROM [datahouse].[dbo].[View_Price1C_Auto]""", sql_con.engine_for_upload())
rhm_prods = pd.read_sql_query("""
   SELECT [sku]
      ,[label]
      ,[brand]
  FROM [datahouse].[dbo].[plytix_products]""", sql_con.engine_for_upload())
rhm_prods.columns = ['Артикул'] + list(rhm_prods.columns[1:])
merged_data = pd.merge(rhm_prods, plytix_prods, on='Артикул', how='inner')
merged_data.drop_duplicates(subset='Артикул', inplace=True)
merged_data.drop('Brand', axis=1, inplace=True)
merged_data.drop('label', axis=1, inplace=True)
merged_data = merged_data.reset_index(drop=True)

In [ ]:
counter = 0
for item in merged_data["Наименование"]:
    errors_counter = 0
    while True:
        try:
            parse(item, clear = counter % 100 == 0)
            break
        except Exception as e:
            if errors_counter == 1:
                print(f'Два раза парсинг упал с ошибкой {item}')
                break
            errors_counter += 1
    counter += 1
    print(counter)

Работа с позицией SensiDO Cream Color 3 in 1 краска для волос /0
https://www.ozon.ru/product/ammiachnaya-kraska-sensi-do-10-0-sredstvo-dlya-volos-uhod-professionalnyy-professionalnaya-kosmetika-1401126420/
CAPTCHA DETECTED Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="layoutPage"]/div[1]/div[4]/div[3]/div[1]/div[1]/div[2]/div/div[1]/h1"}
  (Session info: chrome=125.0.6422.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF784AF1F52+60322]
	(No symbol) [0x00007FF784A6CEC9]
	(No symbol) [0x00007FF784927EBA]
	(No symbol) [0x00007FF784977676]
	(No symbol) [0x00007FF78497773C]
	(No symbol) [0x00007FF7849BE967]
	(No symbol) [0x00007FF78499C25F]
	(No symbol) [0x00007FF7849BBC80]
	(No symbol) [0x00007FF78499BFC3]
	(No symbol) [0x00007FF784969617]
	(No symbol) [0x00007FF78496A211]
	GetHandleVerifier [0x00007FF78

In [2]:
path_to_dir = r'C:/Users/admin/PycharmProjects/pythonProject/price_parsers/parsers_output/ozon/'
all_files = os.listdir(path_to_dir)
error_counter = 0
all_pages = []
for index, filename in enumerate(all_files):
    try:
        all_pages.append(pd.read_excel(path_to_dir + filename))
    except Exception as e:
        raise (e)
        error_counter += 1

data = pd.concat(all_pages).drop(['Unnamed: 0'],
                                 axis=1)  #.to_excel(f'{str(datetime.today().strftime("%Y-%m-%d"))}.xlsx',index=False)
data['Цена'] = [float(str(cell_val).replace('₽', '').replace(' ', '')) if cell_val is not None else None for cell_val in
                data['Цена']]
data['Цена со скидкой'] = [float(str(cell_val).replace('₽', '').replace(' ', '')) if not pd.isna(cell_val) else None for
                           cell_val in data['Цена со скидкой']]

ValueError: could not convert string to float: '1\u2009190\u2009'

In [4]:

merged_data.columns = list(merged_data.columns)[:-1] + ['Название товара наше']
new_data = pd.merge(data, merged_data, on='Название товара наше')
new_data.drop('Бренд', axis=1)
new_data.to_excel('final_data_ozon.xlsx')